#### Symbolic Propagation fingerprint

SPF stands for Symbolic Propagation fingerprint, described [here](Symbolic_propagation_fingerprint.md)

Calibration part is done [here](SPF_Calibration.ipynb)

the calibration output is in data/spf_calibration/calib1.json

In [1]:
import json
import numpy as np
import pandas as pd
import seaborn as sns
from API import API
from secret import API_KEY,MAC_WEAR
import plotly.graph_objects as go

from func.DataLoader import gdfLoader,ReadMarkerMap,RSSIDataLoader

In [2]:
expectation=json.loads(open('data/spf_calibration/calib1.json').read())

In [3]:
# here we get the actual data we want to predict position in zero shot
start,end="2023-06-23 10:42:00.000000","2023-06-23 12:50:10.000000"# should be 4A439

Gatien_API=API(API_KEY)
hour_correction=2
start=pd.to_datetime(start)-pd.Timedelta(hour_correction, unit="h")
end=pd.to_datetime(end)-pd.Timedelta(hour_correction, unit="h")
data=Gatien_API.getRawDataForCartoWear(MAC_WEAR,start,end)
data=pd.DataFrame(data)
data["timestamp"]=pd.to_datetime(data['timestamp'])
data


,timestamp,macModule,rssi
0,2023-06-23 10:50:09.238,A8032A311FAA,-67
1,2023-06-23 10:50:08.643,A8032A311F96,-64
2,2023-06-23 10:50:08.436,A8032A31204A,-74
3,2023-06-23 10:50:06.047,A8032A311F96,-64
4,2023-06-23 10:50:05.870,A8032A31204A,-74
...,...,...,...
11697,2023-06-23 08:42:03.619,A8032A311FAA,-66
11698,2023-06-23 08:42:02.726,A8032A311F96,-65
11699,2023-06-23 08:42:02.400,A8032A31204A,-73
11700,2023-06-23 08:42:01.377,A8032A311F6A,-55


data treatment

In [4]:
data_treated=data.groupby("macModule").mean().reset_index()
data_treated=data_treated[["macModule","rssi"]]
data_treated

,macModule,rssi
0,A8032A311DAA,-71.752879
1,A8032A311F6A,-54.549628
2,A8032A311F96,-64.051635
3,A8032A311FAA,-67.128875
4,A8032A31204A,-72.512910
5,A8032A31204E,-79.503448


In [5]:

room_dict={}
for index,row in data_treated.iterrows():
    room_value=expectation[row["macModule"]]
    for room in room_value:
        if room_value[room]:
            min,max=room_value[room]
            if min<=row["rssi"]<=max:
                room_dict[room]=room_dict.get(room,0)+1
room_dict

{'21855': 1, '21279': 2, '21821': 1, '21820': 3}